In [1]:
!pip install pykrx

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pykrx import stock
from pykrx import bond



In [3]:
import json

def get_market_ohlcv(start_date, end_date, ticker):
    stock_name = stock.get_market_ticker_name(ticker)
    df = stock.get_market_ohlcv(start_date, end_date, ticker)
    df['종목명'] = [stock_name] * len(df)

    return json.dumps(df.to_dict(orient='records'), ensure_ascii=False)

In [4]:
print(get_market_ohlcv("20220720", "20220810", "005930"))

[{"시가": 61800, "고가": 62100, "저가": 60500, "종가": 60500, "거래량": 16782238, "등락률": -0.6568144499178982, "종목명": "삼성전자"}, {"시가": 61100, "고가": 61900, "저가": 60700, "종가": 61800, "거래량": 12291374, "등락률": 2.1487603305785123, "종목명": "삼성전자"}, {"시가": 61800, "고가": 62200, "저가": 61200, "종가": 61300, "거래량": 10261310, "등락률": -0.8090614886731391, "종목명": "삼성전자"}, {"시가": 60900, "고가": 61900, "저가": 60800, "종가": 61100, "거래량": 9193681, "등락률": -0.3262642740619902, "종목명": "삼성전자"}, {"시가": 60800, "고가": 61900, "저가": 60800, "종가": 61700, "거래량": 6597211, "등락률": 0.9819967266775778, "종목명": "삼성전자"}, {"시가": 61300, "고가": 61900, "저가": 61200, "종가": 61800, "거래량": 7320997, "등락률": 0.1620745542949757, "종목명": "삼성전자"}, {"시가": 62300, "고가": 62600, "저가": 61600, "종가": 61900, "거래량": 10745302, "등락률": 0.16181229773462785, "종목명": "삼성전자"}, {"시가": 62400, "고가": 62600, "저가": 61300, "종가": 61400, "거래량": 15093120, "등락률": -0.8077544426494345, "종목명": "삼성전자"}, {"시가": 61000, "고가": 61700, "저가": 60300, "종가": 61300, "거래량": 13154816, "등락률": -0.1628664495114

In [6]:
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_market_ohlcv",
            "description": "특정 종목에 대해 정해진 기간의 주가 데이터를 돌려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "start_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 시작 날짜",
                    },
                    "end_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 종료 날짜",
                    },
                    "ticker": {
                        "type": "string",
                        "description": "6자리 숫자 형식의 종목 티커코드",
                    },
                },
                "required": ["start_date", "end_date", "ticker"],
            },
        }
    }
]


In [7]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

messages= [
  {
    "role": "user",
    "content": "LG전자(066570)의 2022년 8월 1일부터 2022년 12월 1일까지의 주가 데이터를 분석해줘."
  }
]

response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=0,
  max_tokens=1024,
  tools=tools,
  tool_choice="auto" if tools is not None else None,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_G0gG5emCdkDcjjYIhAuh4EGY', function=Function(arguments='{"start_date":"20220801","end_date":"20221201","ticker":"066570"}', name='get_market_ohlcv'), type='function')])


In [8]:
available_functions = {
    "get_market_ohlcv": get_market_ohlcv,
}

for tool_call in response.choices[0].message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        start_date=function_args['start_date'],
        end_date=function_args['end_date'],
        ticker=function_args['ticker']
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "function",
            "name": function_name,
            "content": json.dumps(function_response, ensure_ascii=False)
        }
    )
    
second_response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)
print(second_response.choices[0].message.content)

2022년 8월 1일부터 2022년 12월 1일까지 LG전자의 주가 데이터를 분석한 결과는 다음과 같습니다:

- 8월 1일: 시가 94,800원, 고가 95,300원, 저가 93,600원, 종가 94,400원, 거래량 485,739주, 등락률 0.0%
- 12월 1일: 시가 98,900원, 고가 99,000원, 저가 96,100원, 종가 96,200원, 거래량 456,080주, 등락률 -1.43%

이 기간 동안 LG전자의 주가는 상승 및 하락을 반복하며 등락률이 다양하게 나타났습니다. 특히 8월 31일과 9월 10일에 큰 폭의 상승세를 보였으며, 11월 29일과 12월 1일에 하락세를 보였습니다. 전반적으로 상승하는 추세를 보였지만 중간중간 조정 흐름을 보였습니다.
